# Pockitect Colab Training (Unsloth)

This notebook fine-tunes the model on a GPU-backed Colab runtime using the project training data.

**Before you start:** Runtime → Change runtime type → **GPU**

## 1) Clone the repo

In [5]:
!git clone https://github.com/BrendenKennedy/pockitect-mvp.git
%cd pockitect-mvp

Cloning into 'pockitect-mvp'...
remote: Enumerating objects: 467, done.
remote: Counting objects: 100% (467/467), done.
remote: Compressing objects: 100% (275/275), done.
remote: Total 467 (delta 185), reused 467 (delta 185), pack-reused 0 (from 0)
Receiving objects: 100% (467/467), 495.48 KiB | 2.51 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/pockitect-mvp/pockitect-mvp


## 2) Install dependencies

In [6]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -r requirements.txt

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mtlatoge/unsloth_477437bde7c3489eb5e438e77385ef87
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mtlatoge/unsloth_477437bde7c3489eb5e438e77385ef87
  Resolved https://github.com/unslothai/unsloth.git to commit ab4061e106792fa91e1eba3e4f3d45fa8aba121e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 

## 3) (Optional) Validate training data

In [7]:
!python tools/validate_training_data.py

🔍 Validating 162 training examples...


Validation Summary
Total files:     162
Valid:           162 ✓
Invalid:         0 ✗
Warnings:        0 ⚠️
Errors:          0

✅ All training files are valid!


## 4) (Optional) Regenerate JSONL if you changed training data

In [8]:
!python tools/convert_to_finetuning_format.py --format unsloth

📚 Loaded 162 training pairs

✅ Saved 162 examples to data/finetuning/train_unsloth.jsonl

✅ Conversion complete! Format: unsloth
📁 Output: data/finetuning/train_unsloth.jsonl


## 5) Train

In [9]:
!python tools/finetune_model.py \
    --model qwen2.5-3b \
    --train data/finetuning/train_unsloth.jsonl \
    --epochs 3 \
    --batch-size 2 \
    --early-stopping \
    --early-stopping-patience 3 \
    --eval-steps 50

usage: finetune_model.py [-h] --model MODEL --train TRAIN
                         [--output-dir OUTPUT_DIR] [--preset PRESET]
                         [--max-seq-len MAX_SEQ_LEN] [--epochs EPOCHS]
                         [--batch-size BATCH_SIZE]
                         [--gradient-accumulation GRADIENT_ACCUMULATION]
                         [--lr LR] [--lora-r LORA_R] [--lora-alpha LORA_ALPHA]
                         [--lora-dropout LORA_DROPOUT]
                         [--save-steps SAVE_STEPS]
                         [--logging-steps LOGGING_STEPS]
finetune_model.py: error: unrecognized arguments: --early-stopping --early-stopping-patience 3 --eval-steps 50


## 6) (Optional) Create Ollama model from output

Only if you plan to use Ollama locally after training.

In [ ]:
!ollama create pockitect:finetuned -f data/finetuning/output/Modelfile